# Augment1

In [45]:
import os
import random
from PIL import Image, ImageEnhance, ImageFilter
import numpy

def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = numpy.matrix(matrix, dtype=float)  # Use `float` instead of `numpy.float`
    B = numpy.array(pb).reshape(8)

    res = numpy.linalg.solve(A, B)
    return res.tolist()

def augment_image(image, file_path, augment_cnt):
    # Randomly choose a type of augmentation
    random_augment = random.randint(1, 5)
    augmented_image_path = ""

    if random_augment == 1:
        # 이미지 상하 반전
        print("invert")
        inverted_image = image.transpose(Image.FLIP_TOP_BOTTOM)
        augmented_image_path = os.path.join(file_path, 'inverted_' + str(augment_cnt) + '.jpeg')
        inverted_image.save(augmented_image_path)
            
    elif random_augment == 2:
        # 등변 사다리꼴 모양으로 이미지 변형
        print("skew1")
        width, height = image.size
        
        # 상단 꼭짓점은 그대로 유지하고 하단 꼭짓점은 왼쪽으로 0.15, 오른쪽으로 0.15 만큼 이동
        coeffs = find_coeffs(
            [(0, 0), (width, 0), (width, height), (0, height)],
            [(0, 0), (width, 0), (width + width * 0.15, height), (-width * 0.15, height)]
        )
        
        # 이미지 변형 (하단 부분만 좌우로 늘리기)
        transformed_image = image.transform(
            (width, height),  # 출력 이미지 크기
            Image.PERSPECTIVE,  # 변형 종류
            coeffs,  # 변형 계수
            resample=Image.BICUBIC  # 샘플링 방법
        )
        
        # 변형된 이미지 저장
        augmented_image_path = os.path.join(file_path, 'skew1_' + str(augment_cnt) + '.jpeg')
        transformed_image.save(augmented_image_path)
    
    elif random_augment == 3:
        # 이미지 밝기 조절
        enhancer = ImageEnhance.Brightness(image)
        brightened_img = enhancer.enhance(1.0)
        augmented_image_path = os.path.join(file_path, 'brightened_' + str(augment_cnt) + '.jpeg')
        brightened_img.save(augmented_image_path)
            
    elif random_augment == 4:
        # 이미지 블러 처리
        print("blur")
        blur_strength = 5  # 블러 처리 강도 조절
        blurred_image = image.filter(ImageFilter.BLUR)
        for _ in range(blur_strength):  # 블러 효과를 여러 번 적용하여 강도를 높임
            blurred_image = blurred_image.filter(ImageFilter.BLUR)
        augmented_image_path = os.path.join(file_path, 'blurred_' + str(augment_cnt) + '.jpeg')
        blurred_image.save(augmented_image_path)
        
    elif random_augment == 5:
        # 등변 사다리꼴 모양으로 이미지 변형
        print("skew2")
        width, height = image.size
        
        # 하단 꼭짓점은 그대로 유지하고 상단 꼭짓점은 왼쪽으로 0.15, 오른쪽으로 0.15 만큼 이동
        coeffs = find_coeffs(
            [(0, 0), (width, 0), (width, height), (0, height)],
            [(-width * 0.15, 0), (width + width * 0.15, 0), (width, height), (0, height)]
        )
        
        # 이미지 변형 (상단 부분만 좌우로 늘리기)
        transformed_image = image.transform(
            (width, height),  # 출력 이미지 크기
            Image.PERSPECTIVE,  # 변형 종류
            coeffs,  # 변형 계수
            resample=Image.BICUBIC  # 샘플링 방법
        )
        
        # 변형된 이미지 저장
        augmented_image_path = os.path.join(file_path, 'skew2_' + str(augment_cnt) + '.jpeg')
        transformed_image.save(augmented_image_path)

    return augmented_image_path

# 이미지 폴더 경로
original_image_folder = '"C:\Users\amk25\캐논딥러닝프로젝트\원본 이미지"'
# 증식된 이미지 저장할 폴더 경로
augmented_image_folder = '"C:\Users\amk25\캐논딥러닝프로젝트\증강 이미지"'

# 증식된 이미지 저장할 폴더가 없다면 폴더 생성
if not os.path.exists(augmented_image_folder):
    os.makedirs(augmented_image_folder)
    
# 원본 이미지 폴더에서 파일 목록 가져오기
image_files = os.listdir(original_image_folder)

# 각 이미지에 대해 증강 수행 및 저장
for idx, image_file in enumerate(image_files):
    image_path = os.path.join(original_image_folder, image_file)
    with Image.open(image_path) as img:
        # 3가지의 증강 중 무작위로 2번씩 수행하도록 설정
        for i in range(3):
            augmented_image_path = augment_image(img.copy(), augmented_image_folder, idx * 2 + i)
            # 슬래시(/)를 사용해서 경로를 출력합니다.
            print("Augmented image saved:", augmented_image_path.replace('\\', '/'))

skew2
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/skew2_0.jpeg
skew2
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/skew2_1.jpeg
skew2
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/skew2_2.jpeg
blur
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/blurred_2.jpeg
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/brightened_3.jpeg
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/brightened_4.jpeg
invert
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/inverted_4.jpeg
skew1
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/skew1_5.jpeg
blur
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/blurred_6.jpeg
skew2
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/skew2_6.jpeg
skew1
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Augment/skew1_7.jpeg
Augmented image saved: C:/Users/kimfa/deep learning/규격 미스_Aug